The objective of this notebook is to process bureau.csv and bureau_balance.csv to make Level 2 aggegrated deliquency features. 
The final goal is to have a single row features against each SK_ID_CURR which could be later joined with the application_train.csv so as to form final dataset

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from pyspark.sql import functions as F
from pyspark.sql.window import Window

from pyspark.sql.functions import when, col


In [0]:
# Ignore SettingWithCopyWarning
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

In [0]:
%sql
select * from default.bureau limit 2

In [0]:
%sql
select * from default.bureau_balance limit 2

In [0]:
%sql
select count(distinct sk_id_curr) as customer_count, count(distinct SK_ID_BUREAU) as loan_count from default.bureau

In [0]:
%sql
select count(distinct SK_ID_BUREAU) as loan_count from default.bureau_balance

In [0]:
bu_bal = spark.sql("select * from bureau_balance")
bu_main = spark.sql("select * from bureau")

In [0]:
#dropping duplicates from bu_bal
bu_bal = bu_bal.dropDuplicates()
bu_bal.createOrReplaceTempView("bu_bal")

In [0]:
bu_bal.limit(2).display()

In [0]:

# Define the mapping for STATUS replacement
status_mapping = {
    'C': 'C',
    'X': 0,
    '0': 0,
    '1': 15,
    '2': 45,
    '3': 75,
    '4': 105,
    '5': 135
}

# Start with the original dataframe
bu_bal = bu_bal.withColumn(
    "STATUS",
    when(col("STATUS") == "C", "C")
    .when(col("STATUS") == "X", 0)
    .when(col("STATUS") == "0", 0)
    .when(col("STATUS") == "1", 15)
    .when(col("STATUS") == "2", 45)
    .when(col("STATUS") == "3", 75)
    .when(col("STATUS") == "4", 105)
    .when(col("STATUS") == "5", 135)
    .otherwise(None)  # Set to None (null) for any value not in the mapping
)

In [0]:

# Count occurrences of each unique value in the "STATUS" column
status_counts = bu_bal.groupBy("STATUS").count()

# Display the result
status_counts.show()

In [0]:
def create_raw_dpd_variables(df):

    # Step 1: Filter rows where STATUS is not 'C' and add a dense rank for descending MONTHS_BALANCE within each SK_ID_BUREAU
    window_spec = Window.partitionBy("SK_ID_BUREAU").orderBy(F.desc("MONTHS_BALANCE"))
    df_filtered = (
        df.filter(df['STATUS'] != 'C')
          .withColumn("MOB", F.dense_rank().over(window_spec))
          .filter(F.col("MOB") <= 36)
    )

    # Step 2: Pivot to have each SK_ID_BUREAU as a single row, with MOB values as columns
    bu_bal_2 = df_filtered.groupBy("SK_ID_BUREAU").pivot("MOB", range(1, 37)).agg(F.first("STATUS"))

    # Step 3: Rename columns during the pivot to get dpd_1, dpd_2, ..., dpd_36
    renamed_columns = [F.col(str(mob)).alias(f'dpd_{mob}') for mob in range(1, 37)]
    bu_bal_2 = bu_bal_2.select("SK_ID_BUREAU", *renamed_columns)

    return bu_bal_2

In [0]:
bu_bal_flatten = create_raw_dpd_variables(bu_bal)

In [0]:
bu_bal_flatten.display()
bu_bal_flatten.createOrReplaceTempView("bu_bal_flatten")

In [0]:
%sql
select count(distinct SK_ID_BUREAU) as loan_cout from bu_bal_flatten

In [0]:
bu_main_with_raw_dpd = bu_main.join(bu_bal_flatten, on='SK_ID_BUREAU', how='left')
bu_main_with_raw_dpd.createOrReplaceTempView('bu_main_with_raw_dpd')

In [0]:
%sql
select * from bu_main_with_raw_dpd limit 2

In [0]:
%sql
select
  count(distinct sk_id_curr) as user_count,
  count(distinct sk_id_bureau) as loan_count
from
  bu_main_with_raw_dpd

In [0]:
# # Define loan categories in lowercase
# cash_loans = {
#     'consumer credit', 'mortgage', 'car loan', 'microloan',
#     'loan for working capital replenishment', 'loan for business development',
#     'real estate loan', 'cash loan (non-earmarked)',
#     'loan for the purchase of equipment', 'loan for purchase of shares (margin lending)'
# }

# revolving_loans = {
#     'credit card', 'mobile operator loan', 'interbank credit'
# }

# # Convert 'CREDIT_TYPE' to lowercase and classify using when conditions
# bu_main_with_raw_dpd = bu_main_with_raw_dpd.withColumn('CASH_REV_FLAG',
#     F.when(F.lower(F.col('CREDIT_TYPE')).isin(*cash_loans), 'CASH')
#     .when(F.lower(F.col('CREDIT_TYPE')).isin(*revolving_loans), 'REV')
#     .otherwise('OTHER')
# )

# # Show the result
# bu_main_with_raw_dpd.display()

In [0]:
bu_main_with_raw_dpd.createOrReplaceTempView('bu_main_with_raw_dpd')

In [0]:
%sql
drop table if exists default.bureau_base;
create table default.bureau_base as
select
  *
from
  bu_main_with_raw_dpd;

#### Now that Our Data is ready. We can start creating Dqliquency Features. I have divided features based on Level 1 and Level 2 (based on complexity of aggregations)

## Deliquency LEVEL 2 Variables Creation 

In [0]:
# load the base data 
bu_base_1 = spark.sql("select * from default.bureau_base")
bu_base_1.createOrReplaceTempView("bu_base_1")

In [0]:

bu_base_2 = bu_base_1


mob_values = [3, 6, 9, 12, 15, 18, 24, 36]

# For each MOB value, create a column that counts DPDs greater than 0 for each loan (SK_ID_BUREAU)
for mob in mob_values:
    # Select the dpd columns for the current MOB value (e.g., `dpd_1` to `dpd_36`)
    dpd_cols = [f"dpd_{i}" for i in range(1, mob + 1)]
    
    # Create a new column for each MOB that counts the number of dpd values > 0 for each loan
    bu_base_2 = bu_base_2.withColumn(
        f"COUNT_0P_{mob}MOB",
        sum(F.when(F.col(dpd) > 0, 1).otherwise(0) for dpd in dpd_cols)
    )

# Display or use df2 with the new COUNT_0P_<mob>MOB columns for each loan (SK_ID_BUREAU)
bu_base_2.display()


In [0]:

mob_values = [3, 6, 9, 12, 15, 18, 24, 36]

# # Filter the DataFrame to create separate DataFrames for CASH, REV, and ALL
# df_cash = bu_base_2.filter(F.col("CASH_REV_FLAG") == "CASH")
# df_rev = bu_base_2.filter(F.col("CASH_REV_FLAG") == "REV")
df_all = bu_base_2

# # Define aggregations for each variation of DEQ_MAX_COUNT_DPD0P_{X}MOB
# aggregations_cash = {}
# aggregations_rev = {}
aggregations_all = {}

for mob in mob_values:
    # Define the aggregation for each MOB, calculating the max of COUNT_0P_{mob}MOB
    # aggregations_cash[f"DEQ_MAX_COUNT_DPD0P_{mob}MOB_CASH"] = F.max(F.col(f"COUNT_0P_{mob}MOB"))
    # aggregations_rev[f"DEQ_MAX_COUNT_DPD0P_{mob}MOB_REV"] = F.max(F.col(f"COUNT_0P_{mob}MOB"))
    aggregations_all[f"DEQ_MAX_COUNT_DPD0P_{mob}MOB_ALL"] = F.max(F.col(f"COUNT_0P_{mob}MOB"))

# Apply the aggregations to each filtered DataFrame

# df_aggregated_cash = df_cash.groupBy("SK_ID_CURR").agg(
#     *[aggregation.alias(name) for name, aggregation in aggregations_cash.items()]
# )

# df_aggregated_rev = df_rev.groupBy("SK_ID_CURR").agg(
#     *[aggregation.alias(name) for name, aggregation in aggregations_rev.items()]
# )

df_aggregated_all = df_all.groupBy("SK_ID_CURR").agg(
    *[aggregation.alias(name) for name, aggregation in aggregations_all.items()]
)

#Join all three aggregated DataFrames on SK_ID_CURR to get the final combined result
# bu_features_1 = df_aggregated_cash.join(df_aggregated_rev, on="SK_ID_CURR", how="outer") \
#                              .join(df_aggregated_all, on="SK_ID_CURR", how="outer")

bu_features_1 = df_aggregated_all


In [0]:
mob_values = [3, 6, 9, 12, 15, 18, 24, 36]

# Filter the DataFrame to create separate DataFrames for CASH, REV, and ALL
# df_cash = bu_base_2.filter(F.col("CASH_REV_FLAG") == "CASH")
# df_rev = bu_base_2.filter(F.col("CASH_REV_FLAG") == "REV")
df_all = bu_base_2

# Define aggregations for each variation of DEQ_SUM_COUNT_DPD0P_{X}MOB
# aggregations_cash = {}
# aggregations_rev = {}
aggregations_all = {}

for mob in mob_values:
    # Define the aggregation for each MOB, calculating the sum of COUNT_0P_{mob}MOB
    # aggregations_cash[f"DEQ_SUM_COUNT_DPD0P_{mob}MOB_CASH"] = F.sum(F.col(f"COUNT_0P_{mob}MOB"))
    # aggregations_rev[f"DEQ_SUM_COUNT_DPD0P_{mob}MOB_REV"] = F.sum(F.col(f"COUNT_0P_{mob}MOB"))
    aggregations_all[f"DEQ_SUM_COUNT_DPD0P_{mob}MOB_ALL"] = F.sum(F.col(f"COUNT_0P_{mob}MOB"))

# Apply the aggregations to each filtered DataFrame

# df_aggregated_cash = df_cash.groupBy("SK_ID_CURR").agg(
#     *[aggregation.alias(name) for name, aggregation in aggregations_cash.items()]
# )

# df_aggregated_rev = df_rev.groupBy("SK_ID_CURR").agg(
#     *[aggregation.alias(name) for name, aggregation in aggregations_rev.items()]
# )

df_aggregated_all = df_all.groupBy("SK_ID_CURR").agg(
    *[aggregation.alias(name) for name, aggregation in aggregations_all.items()]
)

#Join all three aggregated DataFrames on SK_ID_CURR to get the final combined result
# bu_features_2 = df_aggregated_cash.join(df_aggregated_rev, on="SK_ID_CURR", how="outer") \
#                              .join(df_aggregated_all, on="SK_ID_CURR", how="outer")

bu_features_2 = df_aggregated_all


In [0]:
mob_values = [3, 6, 9, 12, 15, 18, 24, 36]

# Filter the DataFrame to create separate DataFrames for CASH, REV, and ALL
# df_cash = bu_base_2.filter(F.col("CASH_REV_FLAG") == "CASH")
# df_rev = bu_base_2.filter(F.col("CASH_REV_FLAG") == "REV")
df_all = bu_base_2

# Define aggregations for each variation of DEQ_AVG_COUNT_DPD0P_{X}MOB
# aggregations_cash = {}
# aggregations_rev = {}
aggregations_all = {}

for mob in mob_values:
    # Define the aggregation for each MOB, calculating the sum of COUNT_0P_{mob}MOB
    # aggregations_cash[f"DEQ_AVG_COUNT_DPD0P_{mob}MOB_CASH"] = F.avg(F.col(f"COUNT_0P_{mob}MOB"))
    # aggregations_rev[f"DEQ_AVG_COUNT_DPD0P_{mob}MOB_REV"] = F.avg(F.col(f"COUNT_0P_{mob}MOB"))
    aggregations_all[f"DEQ_AVG_COUNT_DPD0P_{mob}MOB_ALL"] = F.avg(F.col(f"COUNT_0P_{mob}MOB"))

# Apply the aggregations to each filtered DataFrame

# df_aggregated_cash = df_cash.groupBy("SK_ID_CURR").agg(
#     *[aggregation.alias(name) for name, aggregation in aggregations_cash.items()]
# )

# df_aggregated_rev = df_rev.groupBy("SK_ID_CURR").agg(
#     *[aggregation.alias(name) for name, aggregation in aggregations_rev.items()]
# )

df_aggregated_all = df_all.groupBy("SK_ID_CURR").agg(
    *[aggregation.alias(name) for name, aggregation in aggregations_all.items()]
)

#Join all three aggregated DataFrames on SK_ID_CURR to get the final combined result
# bu_features_3 = df_aggregated_cash.join(df_aggregated_rev, on="SK_ID_CURR", how="outer") \
#                              .join(df_aggregated_all, on="SK_ID_CURR", how="outer")

bu_features_3 = df_aggregated_all

In [0]:
mob_values = [3, 6, 9, 12, 15, 18, 24, 36]

# Filter the DataFrame to create separate DataFrames for Active Loans
df_active = bu_base_2.filter(F.col("CREDIT_ACTIVE") == "Active")
df_closed = bu_base_2.filter(F.col("CREDIT_ACTIVE") == "Closed")

# Define aggregations for each variation of DEQ_MAX_COUNT_DPD0P_{X}MOB
aggregations_active = {}
aggregations_closed = {}


for mob in mob_values:
    # Define the aggregation for each MOB, calculating the max of COUNT_0P_{mob}MOB
    aggregations_active[f"DEQ_MAX_COUNT_DPD0P_{mob}MOB_ACTIVE_LOANS"] = F.max(F.col(f"COUNT_0P_{mob}MOB"))
    aggregations_closed[f"DEQ_MAX_COUNT_DPD0P_{mob}MOB_CLOSED_LOANS"] = F.max(F.col(f"COUNT_0P_{mob}MOB"))
    

# Apply the aggregations to each filtered DataFrame

df_aggregated_active = df_active.groupBy("SK_ID_CURR").agg(
    *[aggregation.alias(name) for name, aggregation in aggregations_active.items()]
)

df_aggregated_closed = df_closed.groupBy("SK_ID_CURR").agg(
    *[aggregation.alias(name) for name, aggregation in aggregations_closed.items()]
)

#Join all 2 aggregated DataFrames on SK_ID_CURR to get the final combined result
bu_features_4 = df_aggregated_active
bu_features_5 = df_aggregated_closed


In [0]:
bu_features_level2 = bu_features_1.join(bu_features_2, on="SK_ID_CURR", how = "outer").join(bu_features_3,on="SK_ID_CURR", how = "outer").join(bu_features_4,on="SK_ID_CURR", how = "outer").join(bu_features_5,on="SK_ID_CURR", how = "outer")

In [0]:
bu_features_level2.display()

In [0]:
bu_features_level2.createOrReplaceTempView("bu_features_level2")

In [0]:
%sql
drop table if exists default.bu_features_level2;
create table default.bu_features_level2 as
select
  *
from
  bu_features_level2;

### Done with Creating Deliquency Based features on OFF US Data